In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
from gensim.models import KeyedVectors

In [3]:
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

Found 3000000 word vectors of word2vec


In [4]:
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300

from numpy import linalg as LA
LA.norm(word2vec.word_vec('yes') - word2vec.word_vec('no'))

2.4503

In [5]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.models import Model

Using TensorFlow backend.


In [6]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [7]:
X_train = pd.read_csv("data/train_inter.csv")
X_test = pd.read_csv("data/test_inter.csv")


In [8]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

#X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
#X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())
#X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
#X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [9]:
train_texts1 = []
train_texts2 = []
for i in X_train.question1:
    train_texts1.append(i)
for i in X_train.question2:
    train_texts2.append(i)
    
test_texts1 = []
test_texts2 = []
for i in X_test.question1:
    test_texts1.append(i)
for i in X_test.question2:
    test_texts2.append(i)

In [10]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(train_texts1 + train_texts2 + test_texts1 + test_texts2)


In [11]:
train_sequences_1 = tokenizer.texts_to_sequences(train_texts1)
train_sequences_2 = tokenizer.texts_to_sequences(train_texts2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts2)


train_sequences_1 = pad_sequences(train_sequences_1, maxlen=30)
train_sequences_2 = pad_sequences(train_sequences_2, maxlen=30)
test_sequences_1 = pad_sequences(test_sequences_1, maxlen=30)
test_sequences_2 = pad_sequences(test_sequences_2, maxlen=30)

In [12]:
len(tokenizer.word_index)

137042

In [13]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [14]:
embedding_matrix.shape

(137043, 300)

In [15]:
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=30,
                            trainable=False)

In [16]:
train_sequences_2.shape
y_train.shape

(404290, 1)

In [17]:
from keras.layers.merge import concatenate
lstm_layer = LSTM(200, dropout=0.2, recurrent_dropout=0.2)

sequence_1_input = Input(shape=(30,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(30,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
x2 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, x2])

merged = Dense(100, activation='relu')(merged)
merged = Dropout(0.1)(merged)
preds = Dense(1, activation='sigmoid')(merged)


In [ ]:
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])

In [ ]:
model.fit([train_sequences_1, train_sequences_2], y_train.values, epochs=5, batch_size=50000, shuffle=True, validation_split=0.3)

Train on 283003 samples, validate on 121287 samples
Epoch 1/5


In [62]:
tmp = model.predict([train_sequences_1, train_sequences_2])

In [64]:
tmp.shape

(404290, 400)

In [68]:
X_train = X_train.ix[:,-2:]

In [70]:
tmp = pd.DataFrame(tmp)

In [72]:
X_train = pd.concat([X_train, tmp], axis=1)

In [74]:
X_train.to_csv('data/train_emb.csv' ,index=None)

In [31]:
model = Sequential()
model.add(embedding_layer)
#model.add(Flatten())

model1 = Sequential()
model1.add(embedding_layer)
#model1.add(Flatten())

#merged = concatenate([model.layers[1], model1.layers[1]])

#model.add(Dense(1, activation='sigmoid'))
#model.layers[0].trainable=False
#model.compile(loss='binary_crossentropy', optimizer='rmsprop')
#model.fit(train_sequences_2, y=y_train.values, batch_size=50000, nb_epoch=5, verbose=1, validation_split=0.2, shuffle=True)

In [32]:
tmp = model.predict(train_sequences_1)

In [44]:
train_sequences_1[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    2,    3,    1, 1258,   58,
       1258,    8,  581,    7,  764,  371,    7,   34], dtype=int32)

In [45]:
tmp[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.15625   , -0.08789062, -0.22949219, ...,  0.14160156,
         0.04736328, -0.0135498 ],
       [ 0.0703125 ,  0.08691406,  0.08789062, ..., -0.04760742,
         0.01446533, -0.0625    ],
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469]], dtype=float32)

In [143]:
train_sequences_1.shape
y_train.shape

(404290, 1)

In [97]:
#lstm_layer = LSTM(30)
lstm_layer = LSTM(30, dropout=0.15, recurrent_dropout=0.15)


In [101]:
sequence_1_input = Input(shape=(30,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(30,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

In [102]:
merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

ValueError: zero-dimensional arrays cannot be concatenated

In [15]:
flow = (list(range(1,10,1)) + list(range(10,1,-1)))*1000  
pdata = pd.DataFrame({"a":flow, "b":flow})  
pdata.b = pdata.b.shift(9)  
data = pdata.iloc[10:] * random()  # some noise  

TypeError: 'module' object is not callable

In [ ]:
data